## Imports and inital setup

In [9]:
# Imports
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf


from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

#import helperfunctions as hf

SEED = 42
NUM_EPOCHS = 10

# File path variables
# please make sure to use the correct path to the meta data file

FILEPATH_JPGS = './../data/jpgs/'
FILEPATH_PROCESSED="./../data/processed/"
FILEPATH_OUTPUT = './../data/jpgs/'  # Replace with your folder path

TARGET_LABEL="dx_binary"
BALANCE_LABEL="dx"
IMAGE_SIZE = (224, 224)

In [10]:
from PIL import Image
import numpy as np

def load_and_preprocess_image(image_path, target_size=IMAGE_SIZE):
    # Load the image using PIL
    image = Image.open(image_path)
    
    # Resize the image to the target size
    image = image.resize(target_size)
    
    # Convert the PIL image to a NumPy array
    image_array = np.array(image)
    
    # Perform any additional preprocessing steps here
    # For example, you can normalize pixel values to [0, 1]
    # image_array = image_array / 255.0
    
    return image_array

### Loading (augmented) metadata as test, train, validation from files

In [11]:
# Read the metadata file
train_df = pd.read_csv(FILEPATH_PROCESSED+"train_from_Metadata_processed.csv")
validation_df = pd.read_csv(FILEPATH_PROCESSED+"validation_from_Metadata_processed.csv")
test_df = pd.read_csv(FILEPATH_PROCESSED+"test_from_Metadata_processed.csv")

train_df.sample(15)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,dx_binary,image_path
8897,HAM_0007227,aug_AKZKjoISIC_0029891.jpg,df,histo,75.0,male,lower extremity,rosendahl,not_skin_cancer,./../data/jpgs/aug_AKZKjoISIC_0029891.jpg
8230,HAM_0005918,aug_2K1BEeISIC_0029177.jpg,df,consensus,35.0,female,lower extremity,vidir_molemax,not_skin_cancer,./../data/jpgs/aug_2K1BEeISIC_0029177.jpg
1961,HAM_0005490,ISIC_0033931.jpg,mel,histo,70.0,male,upper extremity,vidir_modern,skin_cancer,./../data/jpgs/ISIC_0033931.jpg
6395,HAM_0004536,aug_q7s3JdISIC_0030314.jpg,bcc,histo,40.0,female,back,vidir_modern,skin_cancer,./../data/jpgs/aug_q7s3JdISIC_0030314.jpg
7172,HAM_0003216,aug_jpsEQbISIC_0034223.jpg,bcc,histo,85.0,female,face,vidir_modern,skin_cancer,./../data/jpgs/aug_jpsEQbISIC_0034223.jpg
1837,HAM_0003558,ISIC_0030129.jpg,mel,histo,55.0,male,back,vidir_modern,skin_cancer,./../data/jpgs/ISIC_0030129.jpg
7621,HAM_0005518,aug_jM3OlvISIC_0032410.jpg,df,histo,30.0,male,lower extremity,vidir_modern,not_skin_cancer,./../data/jpgs/aug_jM3OlvISIC_0032410.jpg
5559,HAM_0005650,aug_8bJm1iISIC_0030387.jpg,akiec,histo,65.0,male,lower extremity,rosendahl,skin_cancer,./../data/jpgs/aug_8bJm1iISIC_0030387.jpg
2309,HAM_0002732,aug_fDNMUJISIC_0034022.jpg,mel,histo,65.0,male,chest,vidir_modern,skin_cancer,./../data/jpgs/aug_fDNMUJISIC_0034022.jpg
4619,HAM_0002705,ISIC_0025089.jpg,akiec,histo,65.0,male,scalp,rosendahl,skin_cancer,./../data/jpgs/ISIC_0025089.jpg


## Setting up the image data generator for training and validation

In [12]:
# Setting up the Image Data Generator for the train data set - including augmentation

datagen_train = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Rescale pixel values to [0, 1], important for CNNs to perform better, deactivate to see images down below
)

datagen_validation = ImageDataGenerator(
    rescale=1.0 / 255.0 #see above
)

validation_generator = datagen_validation.flow_from_dataframe(
    dataframe=validation_df,
    directory=FILEPATH_JPGS,
    x_col="image_id",
    y_col=TARGET_LABEL,
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=32
)

train_data_generator = datagen_train.flow_from_dataframe(
        dataframe=train_df,
        directory=FILEPATH_JPGS,
        x_col="image_id",
        y_col=TARGET_LABEL,
        class_mode="categorical",
        target_size=IMAGE_SIZE,
        batch_size=32
)

Found 2003 validated image filenames belonging to 2 classes.
Found 10506 validated image filenames belonging to 2 classes.


# Adding an neural network model to test the data

## Simple Model Architecture

Source: https://www.tensorflow.org/tutorials/images/cnn (with minor changes)

In [13]:
# Define super basic neural network architecture
model = tf.keras.models.Sequential([
    # Input layer
    tf.keras.layers.InputLayer(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),

    # Convolutional block 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Convolutional block 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Convolutional block 3
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),

    # Flattening the output of the convolutional layers
    tf.keras.layers.Flatten(),

    # Dense (fully connected) layers
    tf.keras.layers.Dense(64, activation='relu'),

    # Output layer
    tf.keras.layers.Dense(train_df[TARGET_LABEL].nunique(), activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 173056)            0

## Model Training

In [14]:
# Train the model

history = model.fit(
    train_data_generator,
    epochs=NUM_EPOCHS,              
    verbose=1,                      # Adjust verbosity level
    batch_size=None,                # Set the batch size, default is 32, can be increased to speed up training
    callbacks=None,                 # List of callbacks to apply during training 
    validation_split=0.0,           # not needed as we use a validation data generator
    validation_data=validation_generator,
    shuffle=True,                   # Shuffle the training data before each epoch
    sample_weight=None,             # Set the weights for the train data set !
    class_weight=None,              # Set the weights for the classes, not needed if we use sample weights
    initial_epoch=0,                # Use this to continue training from a specific epoch
    steps_per_epoch=None,           # Set the number of steps per epoch, default is len(x_train) // batch_size
    validation_steps=None,          # Set the number of steps for validation, default is len(x_val) // batch_size
    validation_batch_size=None,     # Set the batch size for validation, default is batch_size
    validation_freq=1,              # Only relevant if validation data is a generator. Set the frequency to validate the model on the validation set
    max_queue_size=10,              # Set the max size for the generator queue
    workers=-1,                     # Set the max number of processes to generate the data in parallel, -1 means all CPUs
    use_multiprocessing=False       # Set to True if you use a generator in parallel, e.g. model.predict_generator()
)

Epoch 1/10
 76/329 [=====>........................] - ETA: 3:01 - loss: 0.7259 - accuracy: 0.5768

KeyboardInterrupt: 

## Plotting accuracy and loss of train and validation set

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')


# load test images from test_df
test_images = []
test_labels = []

for i in range(test_df.shape[0]):
    image_path = test_df.iloc[i]['image_path']
    img = load_and_preprocess_image(image_path)
    # convert img to np array
    img = np.array(img)
    test_images.append(img)
    test_labels.append(test_df.iloc[i][TARGET_LABEL])


## Evaluating the model on the test set

In [ ]:
from keras.utils import to_categorical

# using label encoder to get the labels

le = LabelEncoder()
test_labels = le.fit_transform(test_df[TARGET_LABEL])

# converting to one hot format / categorial
test_labels = to_categorical(test_labels)

# reshaping the test images
#test_images = np.array(test_images).reshape(-1, 128, 128, 3)
test_images = np.array(test_images).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

## Saving the model

In [ ]:
from datetime import datetime
timestamp = datetime.now()
model_path = f"../models/model_{timestamp}.keras"
model.save(model_path)